<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-11 13:30:15
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: -51.90 K (-0.37%)
Current PnL: -28.75 L (-18.34%)
CY Booked + Current PnL: -14.01 L (-8.94%)
-------------------
Total profit:  1.60 L
Total loss:  -30.35 L
-------------------
Total Booked + Current PnL: 12.97 L (9.99%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.47%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.13 L (66.88%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.25,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,59.0,H-SC,3.43,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
18,COALINDIA,-2.25,3.59,15.09,19.23,23520.0,5402.0,155862.0,484.83,21.35,48.0,L-LC,12.22,182.0,0.23,1.11,25.63,XY25,ATH,MINING
77,TTKPRESTIG,-0.01,-23.97,31.65,0.09,24251.0,-24155.0,76622.0,770.00,77.80,49.0,M-SC,8.47,253.0,-1.00,0.54,5.45,OX40N,NTT,DURABLES
5,ANGELONE,-0.03,13.93,9.01,24.20,31335.0,42525.0,347775.0,3033.00,62.46,62.0,X-SC,3.08,99.0,1.36,2.47,45.65,X40N,NTT,FINANCE
36,ICICIGI,0.73,1.90,17.12,19.34,34908.0,3793.0,203902.0,2252.93,-19.56,60.0,X-MC,3.55,71.0,0.11,1.45,18.14,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WHIRLPOOL,3.24,-26.91,148.90,81.92,124933.0,-30894.0,83904.0,2270.0,-57.25,65.0,M-SC,31.76,218.0,-0.25,0.60,17.86,XR,NTT,DURABLES
68,STARHEALTH,3.01,-6.56,57.61,47.26,151324.0,-18453.0,262670.0,761.0,27.43,67.0,H-SC,1.06,174.0,-0.12,1.87,40.73,XY24,NTT,INSURANCE
45,JCHAC,2.14,-36.43,57.28,-0.01,46540.0,-46555.0,81250.0,2282.0,16530.43,57.0,M-SC,17.03,235.0,-1.00,0.58,12.65,OX40N,BTT,AC
28,GLAXO,1.98,7.38,29.59,39.15,63318.0,14700.0,213984.0,3466.2,-13.83,70.0,X-MC,13.45,62.0,0.23,1.52,35.56,X40,ATH,PHARMA
75,TMPV,1.65,-20.97,55.60,22.98,99264.0,-47365.0,178533.0,600.0,-18.19,69.0,X-LC,3.56,2.0,-0.48,1.27,14.13,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,-3.13,-13.43,68.14,45.57,116290.0,-26469.0,170663.0,844.0,3700.00,70.0,M-SC,7.13,245.0,-0.23,1.21,29.15,XY24,NTT,AUTO
54,QUESS,-2.99,-29.05,101.34,42.84,46737.0,-18887.0,46119.0,424.0,-53.72,55.0,X-SC,20.29,83.0,-0.40,0.33,10.94,XY24,NTT,MISC
76,TRIDENT,-2.66,-28.35,79.37,28.51,52461.0,-26158.0,66097.0,48.0,-15.26,49.0,M-SC,8.89,226.0,-0.50,0.47,13.10,XR,NTT,TEXTILES
80,VALIANTORG,-2.47,-43.27,206.96,74.15,192100.0,-70785.0,92820.0,838.0,-267.19,61.0,H-SC,25.26,145.0,-0.37,0.66,18.98,XR,NTT,CHEMICALS
70,SYMPHONY,-2.26,-28.34,39.51,-0.03,48453.0,-48505.0,122636.0,1306.0,-42.47,58.0,M-SC,10.86,206.0,-1.00,0.87,13.33,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.73,3.97,99.98,107.91,153468.0,5855.0,153499.0,1641.55,-15.28,68.0,M-SC,10.53,220.0,0.04,1.09,15.48,OX40N,ATH,CABLES
66,SIS,-0.82,-20.65,52.82,21.26,46535.0,-22930.0,88102.0,528.00,2082.90,64.0,H-SC,4.20,163.0,-0.49,0.63,21.46,OX40N,NTT,MISC
77,TTKPRESTIG,-0.01,-23.97,31.65,0.09,24251.0,-24155.0,76622.0,770.00,77.80,49.0,M-SC,8.47,253.0,-1.00,0.54,5.45,OX40N,NTT,DURABLES
72,TATAELXSI,-2.14,-24.11,73.97,32.03,74010.0,-31781.0,100054.0,9161.00,-17.88,45.0,H-SC,6.14,157.0,-0.43,0.71,12.07,OX40N,NTT,IT
40,INDIGOPNTS,-1.72,-27.99,38.90,0.02,48897.0,-48860.0,125699.0,1408.00,91.96,33.0,M-SC,6.57,240.0,-1.00,0.89,9.34,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.25,3.59,15.09,19.23,23520.0,5402.0,155862.0,484.83,21.35,48.0,L-LC,12.22,182.0,0.23,1.11,25.63,XY25,ATH,MINING
50,MASFIN,-1.25,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,59.0,H-SC,3.43,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
26,FINCABLES,-0.73,3.97,99.98,107.91,153468.0,5855.0,153499.0,1641.55,-15.28,68.0,M-SC,10.53,220.0,0.04,1.09,15.48,OX40N,ATH,CABLES
1,ABB,-1.14,6.47,36.72,45.57,102281.0,16925.0,278544.0,7934.00,-31.59,70.0,H-MC,3.40,121.0,0.17,1.98,23.69,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-1.14,6.47,36.72,45.57,102281.0,16925.0,278544.0,7934.00,-31.59,70.0,H-MC,3.40,121.0,0.17,1.98,23.69,AR,NTT,ELECTRICAL
50,MASFIN,-1.25,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,59.0,H-SC,3.43,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
26,FINCABLES,-0.73,3.97,99.98,107.91,153468.0,5855.0,153499.0,1641.55,-15.28,68.0,M-SC,10.53,220.0,0.04,1.09,15.48,OX40N,ATH,CABLES
39,INDIAMART,-0.97,-4.22,115.82,106.72,136826.0,-5199.0,118137.0,4810.62,-52.54,51.0,H-SC,7.76,138.0,-0.04,0.84,18.18,AR,ATH,MISC
84,WIPRO,-0.62,-5.65,82.58,72.27,132027.0,-9570.0,159878.0,420.00,-19.24,36.0,M-LC,4.42,101.0,-0.07,1.14,2.77,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.87,-19.00,49.57,21.14,91275.0,-43206.0,184133.0,680.00,-24.66,25.0,X-MC,10.07,75.0,-0.47,1.31,0.00,XY24,NTT,PAINTS
6,ASIANPAINT,0.05,-16.30,44.50,20.95,93777.0,-41034.0,210734.0,3460.25,-17.08,28.0,X-LC,8.07,36.0,-0.44,1.50,13.53,X40,ATH,PAINTS
42,INFY,-1.10,0.85,33.12,34.24,104506.0,2650.0,315538.0,1972.00,-21.39,28.0,X-LC,5.23,6.0,0.03,2.24,9.20,X40,NTT,IT
33,HCLTECH,-1.39,0.67,23.01,23.83,56038.0,1614.0,243538.0,1908.19,5.13,31.0,X-LC,6.52,13.0,0.03,1.73,16.84,X40,ATH,IT
8,AWL,-0.02,-32.00,131.17,57.19,290881.0,-104368.0,221759.0,485.00,-67.84,34.0,X-MC,16.86,58.0,-0.36,1.58,1.06,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-1.83,-19.44,47.15,18.55,143675.0,-73517.0,304720.0,4311.58,-30.74,36.0,X-LC,1.54,3.0,-0.51,2.17,3.67,X40,ATH,IT
81,VBL,-0.24,-8.01,47.48,35.67,137951.0,-25297.0,290545.0,671.64,-18.89,41.0,X-LC,1.63,5.0,-0.18,2.06,4.74,X40N,ATH,FMCG
21,DABUR,0.22,1.83,41.12,43.70,103874.0,4544.0,252612.0,735.00,-6.50,59.0,X-MC,1.94,73.0,0.04,1.80,17.16,XY24,BTT,FMCG
34,HINDUNILVR,0.42,-4.15,18.59,13.67,45806.0,-10663.0,246400.0,2922.00,-11.27,64.0,X-LC,2.51,9.0,-0.23,1.75,16.32,XY25,NTT,FMCG
82,VOLTAS,1.05,16.65,24.47,45.20,54731.0,31923.0,223665.0,1856.00,5.28,64.0,X-MC,2.95,80.0,0.58,1.59,24.58,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.87,-19.00,49.57,21.14,91275.0,-43206.0,184133.0,680.00,-24.66,25.0,X-MC,10.07,75.0,-0.47,1.31,0.00,XY24,NTT,PAINTS
2,ABBOTINDIA,-1.41,-9.73,30.79,18.07,49714.0,-17395.0,161460.0,35195.00,-22.68,40.0,X-MC,9.78,64.0,-0.35,1.15,1.03,X40,ATH,PHARMA
8,AWL,-0.02,-32.00,131.17,57.19,290881.0,-104368.0,221759.0,485.00,-67.84,34.0,X-MC,16.86,58.0,-0.36,1.58,1.06,XY24,NTT,FMCG
53,PGHH,-0.73,-10.92,48.19,32.01,91661.0,-23312.0,190208.0,17616.87,-37.34,44.0,X-MC,8.95,60.0,-0.25,1.35,2.81,X40,ATH,FMCG
44,ITC,-1.23,-22.20,42.38,10.78,79241.0,-53340.0,186978.0,452.00,-53.82,39.0,X-LC,18.86,1.0,-0.67,1.33,2.99,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.99,-29.05,101.34,42.84,46737.0,-18887.0,46119.0,424.00,-53.72,55.0,X-SC,20.29,83.0,-0.40,0.33,10.94,XY24,NTT,MISC
58,RELAXO,-1.03,-52.16,218.35,52.31,151638.0,-75713.0,69447.0,1176.00,-49.09,41.0,X-SC,13.16,92.0,-0.50,0.49,3.27,X40N,NTT,FOOTWEAR
12,BATAINDIA,-1.58,-41.68,131.85,35.20,98935.0,-53634.0,75036.0,2096.00,-1.66,54.0,X-SC,21.00,93.0,-0.54,0.53,6.63,X40,NTT,FOOTWEAR
51,MEDANTA,1.07,-3.85,29.05,24.08,35458.0,-4891.0,122059.0,1486.00,-11.59,54.0,X-SC,11.55,91.0,-0.14,0.87,11.80,XY24,NTT,HEALTHCARE
35,HONAUT,0.09,-20.88,81.15,43.32,104570.0,-34012.0,128860.0,58357.33,-32.00,44.0,X-SC,9.85,90.0,-0.33,0.92,4.63,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.23,-22.20,42.38,10.78,79241.0,-53340.0,186978.0,452.00,-53.82,39.0,X-LC,18.86,1.0,-0.67,1.33,2.99,X40,NTT,FMCG
75,TMPV,1.65,-20.97,55.60,22.98,99264.0,-47365.0,178533.0,600.00,-18.19,69.0,X-LC,3.56,2.0,-0.48,1.27,14.13,XY24,NTT,AUTO
73,TCS,-1.83,-19.44,47.15,18.55,143675.0,-73517.0,304720.0,4311.58,-30.74,36.0,X-LC,1.54,3.0,-0.51,2.17,3.67,X40,ATH,IT
81,VBL,-0.24,-8.01,47.48,35.67,137951.0,-25297.0,290545.0,671.64,-18.89,41.0,X-LC,1.63,5.0,-0.18,2.06,4.74,X40N,ATH,FMCG
42,INFY,-1.10,0.85,33.12,34.24,104506.0,2650.0,315538.0,1972.00,-21.39,28.0,X-LC,5.23,6.0,0.03,2.24,9.20,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.95,-32.17,95.91,32.89,50230.0,-24834.0,52372.0,1800.00,-746.53,55.0,L-MC,11.54,259.0,-0.49,0.37,44.23,XR,NTT,BANKS
67,SONACOMS,1.38,-6.85,49.32,39.09,46474.0,-6930.0,94229.0,804.02,-27.16,69.0,M-SC,3.18,227.0,-0.15,0.67,33.30,AR,ATH,AUTO
50,MASFIN,-1.25,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,59.0,H-SC,3.43,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
14,BSOFT,0.25,-15.40,78.23,50.79,93091.0,-21657.0,118996.0,831.70,8.62,63.0,H-SC,7.71,171.0,-0.23,0.85,38.55,XR,ATH,IT
79,VAIBHAVGBL,-1.37,-24.23,112.17,60.77,155351.0,-44279.0,138496.0,521.00,59.78,57.0,H-SC,3.00,158.0,-0.29,0.98,25.56,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-0.03,13.93,9.01,24.20,31335.0,42525.0,347775.0,3033.00,62.46,62.0,X-SC,3.08,99.0,1.36,2.47,45.65,X40N,NTT,FINANCE
67,SONACOMS,1.38,-6.85,49.32,39.09,46474.0,-6930.0,94229.0,804.02,-27.16,69.0,M-SC,3.18,227.0,-0.15,0.67,33.30,AR,ATH,AUTO
11,BANDHANBNK,1.49,-18.62,136.43,92.42,309288.0,-51858.0,226701.0,400.00,59.48,74.0,H-SC,8.56,169.0,-0.17,1.61,31.02,XY24,NTT,BANKS
1,ABB,-1.14,6.47,36.72,45.57,102281.0,16925.0,278544.0,7934.00,-31.59,70.0,H-MC,3.40,121.0,0.17,1.98,23.69,AR,NTT,ELECTRICAL
7,ATULAUTO,-3.13,-13.43,68.14,45.57,116290.0,-26469.0,170663.0,844.00,3700.00,70.0,M-SC,7.13,245.0,-0.23,1.21,29.15,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.65
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.84
MC    30.71
LC    23.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.69
X40      23.21
X40N     15.20
XR        8.91
AR        8.90
XY25      8.71
OX40N     7.50
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.12
H-SC    22.98
X-LC    20.09
M-SC    11.64
X-SC    10.43
H-MC     4.84
M-MC     1.38
M-LC     1.14
H-LC     1.12
L-LC     1.11
L-SC     0.79
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.98,-12.35,50.10
FINANCE,13.42,-10.49,53.97
IT,12.10,-25.70,88.49
MISC,7.15,-29.32,82.22
ELECTRICAL,6.09,-9.76,49.81
PAINTS,5.03,-30.46,49.84
INSURANCE,4.81,-0.34,34.68
PHARMA,4.08,-2.50,34.76
AUTO,3.15,-18.21,59.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3215585.0,21
AR,1320379.0,10
XR,1238498.0,12
X40,1231973.0,15
X40N,982288.0,10
OX40N,736036.0,10
XY25,392676.0,6
SR,295206.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3561169.0,24
X-MC,1634860.0,16
M-SC,1419219.0,15
X-LC,1063307.0,12
X-SC,823518.0,9
H-MC,397842.0,3
L-SC,180042.0,2
M-LC,132027.0,1
H-LC,76476.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216990.0      6
           AR         935741.0      5
M-SC       XY24       800636.0      6
H-SC       XR         787054.0      7
X-LC       X40        533069.0      6
X-MC       X40        533065.0      7
           XY24       486030.0      3
           X40N       405694.0      4
X-SC       X40N       354388.0      4
H-SC       OX40N      326178.0      4
M-SC       OX40N      321609.0      5
X-SC       XY24       303291.0      3
H-SC       SR         295206.0      2
X-LC       X40N       222206.0      2
X-MC       XY25       210071.0      2
H-MC       XY24       209260.0      1
X-LC       XY24       199378.0      2
H-MC       AR         188582.0      2
M-SC       XR         177394.0      2
X-SC       X40        165839.0      2
M-LC       XR         132027.0      1
M-SC       AR         119580.0      2
X-LC       XY25       108654.0      2
L-SC       XR          91793.0      1
           OX40N       88249.0      1
H-LC       AR          76476.0      1
M-MC       XY25        50431.0      1
L-MC       XR          50230.0      1
L-LC       XY25        23520.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
